<a href="https://colab.research.google.com/github/aakankshalahane/Machine-Learning-Intern-Projects/blob/main/Restaurant_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Restaurant Recommendation
#libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#Data
df = pd.read_csv("/content/Dataset .csv")

In [ ]:
df.shape

(9551, 21)

In [ ]:
display(df.head())

,Restaurant ID,Restaurant Name,Country Code,City,Address,Locality,Locality Verbose,Longitude,Latitude,Cuisines,...,Currency,Has Table booking,Has Online delivery,Is delivering now,Switch to order menu,Price range,Aggregate rating,Rating color,Rating text,Votes
0,6317637,Le Petit Souffle,162,Makati City,"Third Floor, Century City Mall, Kalayaan Avenu...","Century City Mall, Poblacion, Makati City","Century City Mall, Poblacion, Makati City, Mak...",121.027535,14.565443,"French, Japanese, Desserts",...,Botswana Pula(P),Yes,No,No,No,3,4.8,Dark Green,Excellent,314
1,6304287,Izakaya Kikufuji,162,Makati City,"Little Tokyo, 2277 Chino Roces Avenue, Legaspi...","Little Tokyo, Legaspi Village, Makati City","Little Tokyo, Legaspi Village, Makati City, Ma...",121.014101,14.553708,Japanese,...,Botswana Pula(P),Yes,No,No,No,3,4.5,Dark Green,Excellent,591
2,6300002,Heat - Edsa Shangri-La,162,Mandaluyong City,"Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...","Edsa Shangri-La, Ortigas, Mandaluyong City","Edsa Shangri-La, Ortigas, Mandaluyong City, Ma...",121.056831,14.581404,"Seafood, Asian, Filipino, Indian",...,Botswana Pula(P),Yes,No,No,No,4,4.4,Green,Very Good,270
3,6318506,Ooma,162,Mandaluyong City,"Third Floor, Mega Fashion Hall, SM Megamall, O...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.056475,14.585318,"Japanese, Sushi",...,Botswana Pula(P),No,No,No,No,4,4.9,Dark Green,Excellent,365
4,6314302,Sambo Kojin,162,Mandaluyong City,"Third Floor, Mega Atrium, SM Megamall, Ortigas...","SM Megamall, Ortigas, Mandaluyong City","SM Megamall, Ortigas, Mandaluyong City, Mandal...",121.057508,14.584450,"Japanese, Korean",...,Botswana Pula(P),Yes,No,No,No,4,4.8,Dark Green,Excellent,229


In [ ]:
#Missing Value
df['Cuisines'].fillna('Unknown', inplace=True)
df['Price range'].fillna(df['Price range'].median(), inplace=True)
df['Aggregate rating'].fillna(df['Aggregate rating'].mean(), inplace=True)

/tmp/ipython-input-3084858813.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Cuisines'].fillna('Unknown', inplace=True)
/tmp/ipython-input-3084858813.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try usin

In [ ]:
# columns to string
df['Cuisines'] = df['Cuisines'].astype(str)

In [ ]:
# lowercase text
df['Cuisines'] = df['Cuisines'].str.lower()

In [ ]:
#Recommendation
df['content'] = df['Cuisines'] + ' ' + df['Price range'].astype(str)

In [ ]:
#Create Matrix
df['content'] = df['Cuisines'] + ' ' + df['Price range'].astype(str)
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['content'])

print("TF-IDF matrix shape:", tfidf_matrix.shape)

TF-IDF matrix shape: (9551, 149)


In [ ]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

def recommend_restaurants(cuisine_pref, price_pref, top_n=5):
    user_input = f"{cuisine_pref.lower()} {price_pref}"
    user_vec = tfidf.transform([user_input])
    sim_scores = cosine_similarity(user_vec, tfidf_matrix).flatten()

    # Get indices of top recommendations
    top_indices = sim_scores.argsort()[-top_n:][::-1]

    print("\n🍽️ Top Restaurant Recommendations:")
    return df.iloc[top_indices][['Restaurant Name', 'Cuisines', 'Price range', 'Aggregate rating']]

In [ ]:
#Test
user_cuisine = "Italian"
user_price = 3

recommendations = recommend_restaurants(user_cuisine, user_price, top_n=5)
print(recommendations)

# Evaluation
print("\n✅ The recommendation system uses content-based filtering,")
print("   comparing user preferences with restaurant features using TF-IDF & cosine similarity.")
print("   Recommendations are most similar to the cuisine and price entered by the user.")


🍽️ Top Restaurant Recommendations:
     Restaurant Name Cuisines  Price range  Aggregate rating
1106   Da Pizza Zone  italian            2               0.0
31      Villa Tevere  italian            4               4.1
3603         Foodies  italian            1               0.0
9344       San Carlo  italian            3               4.2
3219  28 Capri Italy  italian            3               3.9

✅ The recommendation system uses content-based filtering,
   comparing user preferences with restaurant features using TF-IDF & cosine similarity.
   Recommendations are most similar to the cuisine and price entered by the user.
